In [15]:
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras import layers
import sys
from tensorflow.keras.preprocessing import text

# 모듈 설정
try:
    sys.modules['keras_preprocessing'] = text
    sys.modules['keras.preprocessing'] = text
    sys.modules['keras.preprocessing.text'] = text
    print("✅ 모듈 설정 완료")
except Exception as e:
    print(f"⚠ 모듈 설정 오류: {e}")

# 호환성 문제 해결
class FixedLSTM(LSTM):
    def __init__(self, *args, **kwargs):
        kwargs.pop('time_major', None)
        super().__init__(*args, **kwargs)

def load_model_safely(model_path):
    """모델 안전 로드"""
    try:
        custom_objects = {'LSTM': FixedLSTM, 'Bidirectional': Bidirectional}
        model = load_model(model_path, custom_objects=custom_objects)
        print(f"✅ {model_path} 로드 성공!")
        return model
    except:
        try:
            model = load_model(model_path, compile=False)
            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
            print(f"✅ {model_path} 로드 성공! (재컴파일)")
            return model
        except Exception as e:
            print(f"❌ {model_path} 로드 실패: {e}")
            raise

# 설정
try:
    with open('tokenizer.pickle', 'rb') as f:
        tokenizer = pickle.load(f)
    model = load_model_safely('lstm_model2.h5')
    print("✅ 모든 파일 로드 완료!")
except Exception as e:
    print(f"❌ 파일 로드 실패: {e}")
    exit()

# 라벨 설정
aspects = ['미백', '보습', '트러블', '보호', '노화방지']
polarities = ['긍정', '부정', '중립']
label_cols = [f"{a}_{p}" for a in aspects for p in polarities]
max_len = 100

def explain_threshold_with_example(text):
    """임계값을 쉽게 설명하는 함수"""
    
    print(f"\n{'🎯 임계값(Threshold) 쉬운 설명':=^60}")
    print(f"분석할 텍스트: '{text}'")
    print("=" * 60)
    
    # 예측 수행
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    probs = model.predict(padded, verbose=0)[0]
    
    # 모든 확률값을 보기 좋게 정리
    all_scores = []
    for i, label in enumerate(label_cols):
        all_scores.append((label, probs[i]))
    
    # 확률 높은 순으로 정렬
    all_scores.sort(key=lambda x: x[1], reverse=True)
    
    print("\n📊 모델이 예측한 모든 확률값:")
    print("-" * 40)
    for label, prob in all_scores:
        percentage = prob * 100
        bar = "█" * max(1, int(prob * 20))
        print(f"{label:12s}: {prob:.3f} ({percentage:4.1f}%) {bar}")
    
    print(f"\n🔍 다양한 임계값으로 결과 비교:")
    print("-" * 50)
    
    thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    
    for threshold in thresholds:
        # 임계값 이상인 예측만 선택
        predictions = [label for label, prob in all_scores if prob >= threshold]
        
        print(f"\n임계값 {threshold} ({threshold*100:.0f}%):")
        print(f"  → 예측 개수: {len(predictions)}개")
        if predictions:
            print(f"  → 예측 결과: {predictions[:3]}{'...' if len(predictions) > 3 else ''}")
        else:
            print(f"  → 예측 결과: 없음 (모든 확률이 {threshold*100:.0f}%보다 낮음)")
    
    print(f"\n💡 임계값 선택 가이드:")
    print("  🔸 0.2~0.3: 많은 예측 (민감함, 부정확할 수 있음)")
    print("  🔸 0.4~0.5: 적당한 예측 (균형적)")  
    print("  🔸 0.6~0.7: 확실한 예측만 (보수적, 놓칠 수 있음)")

def threshold_comparison_simple(text):
    """임계값 비교를 단순하게"""
    
    print(f"\n{'🎮 임계값 게임 - 직접 확인해보세요!':=^60}")
    print(f"텍스트: '{text}'")
    
    # 예측
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    probs = model.predict(padded, verbose=0)[0]
    
    scores = {label_cols[i]: probs[i] for i in range(len(label_cols))}
    
    print(f"\n🎯 임계값별 결과 한눈에 비교:")
    
    # 테이블 헤더
    print(f"{'라벨':15s} {'확률':>8s} {'0.3':>5s} {'0.4':>5s} {'0.5':>5s} {'0.6':>5s}")
    print("-" * 50)
    
    # 확률 높은 순으로 상위 10개만
    top_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:10]
    
    for label, prob in top_scores:
        # 각 임계값에서 예측되는지 확인
        check_03 = "✅" if prob >= 0.3 else "❌"
        check_04 = "✅" if prob >= 0.4 else "❌" 
        check_05 = "✅" if prob >= 0.5 else "❌"
        check_06 = "✅" if prob >= 0.6 else "❌"
        
        print(f"{label:15s} {prob:>7.3f} {check_03:>5s} {check_04:>5s} {check_05:>5s} {check_06:>5s}")

def recommend_threshold(text):
    """이 텍스트에 가장 좋은 임계값 추천"""
    
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    probs = model.predict(padded, verbose=0)[0]
    
    scores = {label_cols[i]: probs[i] for i in range(len(label_cols))}
    
    # 다양한 임계값에서 예측 개수 계산
    thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
    results = {}
    
    for threshold in thresholds:
        predictions = [label for label, prob in scores.items() if prob >= threshold]
        results[threshold] = len(predictions)
    
    print(f"\n🏆 '{text}' 에 대한 임계값 추천:")
    print("-" * 40)
    
    for threshold, count in results.items():
        if count == 0:
            status = "😴 너무 엄격함 (예측 없음)"
        elif count <= 3:
            status = "😊 적당함 (좋은 선택!)"
        elif count <= 6:
            status = "😅 많음 (괜찮음)"
        else:
            status = "😵 너무 많음 (너무 관대함)"
            
        print(f"  임계값 {threshold}: {count}개 예측 → {status}")
    
    # 추천 임계값 (2-4개 예측하는 임계값)
    best_threshold = None
    for threshold, count in results.items():
        if 2 <= count <= 4:
            best_threshold = threshold
            break
    
    if best_threshold:
        print(f"\n🎯 추천 임계값: {best_threshold} ← 이걸 사용하세요!")
    else:
        print(f"\n🎯 추천 임계값: 0.4 ← 기본값 사용하세요!")

# 실제 사용 예제
if __name__ == "__main__":
    
    # 테스트 문장들
    samples = [
        "정말 좋아요! 피부가 밝아지고 촉촉해졌어요!",
        "별로예요... 여드름이 더 생겼어요",
        "그냥 무난해요. 특별한 건 없어요."
    ]
    
    print("🚀 임계값 완전정복 가이드")
    
    for i, sample in enumerate(samples, 1):
        print(f"\n{'📝 예제 ' + str(i):=^70}")
        
        # 1. 기본 설명
        explain_threshold_with_example(sample)
        
        # 2. 비교표
        threshold_comparison_simple(sample)
        
        # 3. 추천
        recommend_threshold(sample)
        
        print("\n" + "="*70)
    
    print(f"\n🎉 이제 임계값을 완전히 이해했어요!")
    print(f"💡 팁: 보통 0.3~0.4가 가장 좋은 결과를 줍니다!")

✅ 모듈 설정 완료


✅ lstm_model2.h5 로드 성공!
✅ 모든 파일 로드 완료!
🚀 임계값 완전정복 가이드

================================📝 예제 1================================

===================🎯 임계값(Threshold) 쉬운 설명===================
분석할 텍스트: '정말 좋아요! 피부가 밝아지고 촉촉해졌어요!'

📊 모델이 예측한 모든 확률값:
----------------------------------------
보습_긍정       : 1.000 (100.0%) ███████████████████
트러블_긍정      : 0.009 ( 0.9%) █
미백_긍정       : 0.001 ( 0.1%) █
노화방지_긍정     : 0.000 ( 0.0%) █
보호_긍정       : 0.000 ( 0.0%) █
보습_부정       : 0.000 ( 0.0%) █
트러블_부정      : 0.000 ( 0.0%) █
보습_중립       : 0.000 ( 0.0%) █
노화방지_부정     : 0.000 ( 0.0%) █
트러블_중립      : 0.000 ( 0.0%) █
미백_중립       : 0.000 ( 0.0%) █
노화방지_중립     : 0.000 ( 0.0%) █
보호_중립       : 0.000 ( 0.0%) █
미백_부정       : 0.000 ( 0.0%) █
보호_부정       : 0.000 ( 0.0%) █

🔍 다양한 임계값으로 결과 비교:
--------------------------------------------------

임계값 0.2 (20%):
  → 예측 개수: 1개
  → 예측 결과: ['보습_긍정']

임계값 0.3 (30%):
  → 예측 개수: 1개
  → 예측 결과: ['보습_긍정']

임계값 0.4 (40%):
  → 예측 개수: 1개
  → 예측 결과: ['보습_긍정']

임계값 0.5 (50%):
  → 예측 